In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,transform_alocation_dataframe,base_selection,expand_rows,replicate_row,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler
from streamlit_app.validador import validador

2025-04-25 19:56:47.955 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:56:47.960 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:56:47.965 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:56:47.968 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/CARLOS VIADO 2.xlsx')

professores_raw = pd.read_excel('/home/cayena/Downloads/CARLOS VIADO.xlsx')

In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
Ts = TeacherScheduler(df_result, df_professor)
base_alocada = Ts.schedule_teachers()
base_alocada

2778


,professores_alocados,nome grupo
0,MATTHEW,ABU DHABI EAD - 4X
1,LORRANY,AUCKLAND P - 4X
2,CAROL,BAKERSFIELD P - 4X
3,DARLENE,BELIZE P - 3X
4,MADISON,BERMUDAS P - 4X
...,...,...
139,FELIPE,NORTHAMPTOM P - 3X
140,CARLOS,PHILADELPHIA P - 3X
141,LEONEL,SAN JUAN EAD - 3X
142,CLAUDIO,SCOTT EAD - 3X


In [7]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor


In [9]:
numero_aulas = df_final.groupby('teacher').agg({'n aulas':'sum'}).sort_values('n aulas',ascending=False).reset_index()
numero_aulas = pd.merge(numero_aulas,df_professor[['TEACHER','MEDIA']],left_on='teacher',right_on='TEACHER',how='left')

In [10]:
numero_aulas['MIN_AULAS'] = numero_aulas['MEDIA'] - 3

## soluçoes ##
1 - tentar olhar para o minimo de aulas (botar o minimo para todos) / vai dar errado.
2 - dar um peso grande para quem tem muita aulas (>= 20) e pouco peso para que tem pouca aulas (<=10).
 2.1 - 24 aulas = minimo vai ser 24 - 3 = 21 / 12 aulas = minimo vai ser 12 - 6 = 6 aulas 
3 - pensar em algo para que tente alocar o maximo, o minimo mas se nao der, tudo bem.